# Tutorial 09: Model Serialization

🟡 **Intermediate** — Familiarity with ML concepts helpful

Learn how to save, load, and manage boosters models for deployment.

## What you'll learn

1. Save models with boosters' native binary format
2. Save models as JSON for inspection
3. Load models for inference
4. Model versioning best practices

In [ ]:
import numpy as np
import json
import tempfile
from pathlib import Path
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

import boosters

## Train a Model

In [ ]:
# Generate and train
X, y = make_regression(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create datasets
train_data = boosters.Dataset(X_train, y_train)
test_data = boosters.Dataset(X_test)

# Train model - use max_depth=3 for cleaner tree visualization later
config = boosters.GBDTConfig(n_estimators=100, max_depth=3, learning_rate=0.1)
model = boosters.GBDTModel.train(train_data, config=config)

# Baseline predictions
y_pred_original = model.predict(test_data)

print(f"Model trained with {model.n_trees} trees")
print(f"Predictions shape: {y_pred_original.shape}")

## Save with Native Binary Format

boosters uses a compact binary format (`.bstr`) for efficient model storage:

In [ ]:
# Create temp directory for examples
tmpdir = tempfile.mkdtemp()

# Serialize to binary bytes
model_bytes = model.to_bytes()

# Save to file
binary_path = Path(tmpdir) / "model.bstr"
binary_path.write_bytes(model_bytes)

file_size = binary_path.stat().st_size
print(f"Model saved to: {binary_path}")
print(f"File size: {file_size / 1024:.1f} KB")
print(f"Bytes in memory: {len(model_bytes) / 1024:.1f} KB")

## Load from Binary Format

In [ ]:
# Load model from file
loaded_bytes = binary_path.read_bytes()
loaded_model = boosters.GBDTModel.from_bytes(loaded_bytes)

# Verify predictions match
y_pred_loaded = loaded_model.predict(test_data)

print(f"Loaded model trees: {loaded_model.n_trees}")
print(f"Predictions match: {np.allclose(y_pred_original, y_pred_loaded)}")

## Save as JSON (Human-Readable)

For debugging or interoperability, save as JSON:

In [ ]:
# Serialize to JSON bytes
json_bytes = model.to_json_bytes()

# Save to file
json_path = Path(tmpdir) / "model.bstr.json"
json_path.write_bytes(json_bytes)

json_size = json_path.stat().st_size
print(f"JSON file size: {json_size / 1024:.1f} KB")
print(f"Binary is {json_size / file_size:.1f}x smaller than JSON")

# Peek at the JSON structure
json_preview = json.loads(json_bytes)
print(f"\nJSON keys: {list(json_preview.keys())}")

In [ ]:
# Load from JSON
loaded_json = boosters.GBDTModel.from_json_bytes(json_path.read_bytes())

y_pred_json = loaded_json.predict(test_data)
print(f"JSON model predictions match: {np.allclose(y_pred_original, y_pred_json)}")

## Visualize Tree Structure

Use `plot_tree` to visualize individual trees from a model.
This is useful for debugging and understanding model decisions.
We used `max_depth=3` above to keep the visualization readable:

In [ ]:
import matplotlib.pyplot as plt

# Define feature names for readable visualization
feature_names = [f'feature_{i}' for i in range(10)]

# Visualize first tree
ax = boosters.plot_tree(model, tree_index=0, feature_names=feature_names)
plt.title("Tree 0: First boosting round")
plt.show()

In [ ]:
# Get tree structure as a DataFrame for programmatic analysis
df = boosters.tree_to_dataframe(model, tree_index=0, feature_names=feature_names)

print("Tree structure (first 10 nodes):")
print(df.head(10).to_string())

# Find the most important splits
splits = df[~df['is_leaf']].sort_values('gain', ascending=False)
print(f"\nTop 3 splits by gain:")
for _, row in splits.head(3).iterrows():
    print(f"  {row['feature']} ≤ {row['threshold']:.3f} (gain={row['gain']:.1f}, cover={row['cover']:.0f})")

## Model Metadata

Store metadata alongside your model for versioning:

In [ ]:
import hashlib
from datetime import datetime
from sklearn.metrics import r2_score

def compute_model_signature(model, X_sample):
    """Compute a signature for model verification."""
    predictions = model.predict(boosters.Dataset(X_sample[:10]))
    return hashlib.md5(predictions.tobytes()).hexdigest()

# Calculate R² score
y_pred_flat = y_pred_original.flatten()
original_score = r2_score(y_test, y_pred_flat)

# Create metadata
metadata = {
    "version": "1.0.0",
    "created": datetime.now().isoformat(),
    "model_type": "GBDTModel",
    "n_features": X_train.shape[1],
    "n_trees": model.n_trees,
    "n_samples_trained": X_train.shape[0],
    "metrics": {
        "test_r2": float(original_score),
    },
    "signature": compute_model_signature(model, X_test),
}

# Save metadata alongside model
metadata_path = Path(tmpdir) / "model_metadata.json"
metadata_path.write_text(json.dumps(metadata, indent=2))

print("Model Metadata:")
print(json.dumps(metadata, indent=2))

## Verify Loaded Model

In [ ]:
def verify_model(model, metadata, X_sample):
    """Verify a loaded model matches its metadata."""
    # Check signature
    current_signature = compute_model_signature(model, X_sample)
    if current_signature != metadata["signature"]:
        raise ValueError("Model signature mismatch!")
    
    # Check features
    if X_sample.shape[1] != metadata["n_features"]:
        raise ValueError(f"Expected {metadata['n_features']} features, got {X_sample.shape[1]}")
    
    print("✅ Model verified successfully!")
    print(f"   Version: {metadata['version']}")
    print(f"   Created: {metadata['created']}")
    print(f"   Trees: {metadata['n_trees']}")
    print(f"   Test R²: {metadata['metrics']['test_r2']:.4f}")

# Load and verify
loaded_metadata = json.loads(metadata_path.read_text())
verify_model(loaded_model, loaded_metadata, X_test)

## GBLinear Models

The same API works for GBLinear models:

In [ ]:
# Train a GBLinear model
linear_config = boosters.GBLinearConfig(n_estimators=50, learning_rate=0.5)
linear_model = boosters.GBLinearModel.train(train_data, config=linear_config)

# Save and load
linear_bytes = linear_model.to_bytes()
linear_path = Path(tmpdir) / "linear_model.bstr"
linear_path.write_bytes(linear_bytes)

loaded_linear = boosters.GBLinearModel.from_bytes(linear_path.read_bytes())

# Verify
y_pred_linear_orig = linear_model.predict(test_data)
y_pred_linear_load = loaded_linear.predict(test_data)

print(f"GBLinear model size: {len(linear_bytes) / 1024:.1f} KB")
print(f"Predictions match: {np.allclose(y_pred_linear_orig, y_pred_linear_load)}")

## Cleanup

In [ ]:
# List saved files
print("Saved files:")
for f in sorted(Path(tmpdir).iterdir()):
    print(f"  {f.name}: {f.stat().st_size / 1024:.1f} KB")

# Cleanup
import shutil
shutil.rmtree(tmpdir)
print(f"\nCleaned up: {tmpdir}")

## Best Practices

1. **Use binary format for production** — Smaller files, faster loading
2. **Use JSON for debugging** — Human-readable, inspectable
3. **Always save metadata** — Version, creation date, metrics, signature
4. **Verify after loading** — Check signatures to catch corruption
5. **Version your models** — Use semantic versioning

## Summary

In this tutorial, you learned:

1. ✅ Save models with `.to_bytes()` (compact binary format)
2. ✅ Save models with `.to_json_bytes()` (human-readable)
3. ✅ Load models with `.from_bytes()` and `.from_json_bytes()`
4. ✅ Create and verify model metadata

## Next Steps

- [API Reference](../api/index.rst) — Complete API documentation
- [Tutorial 10: Linear Trees](10-linear-trees.ipynb) — Advanced tree configurations